In [80]:
import cv2
import os
import numpy as np
import itertools
import random

import matplotlib.pyplot as plt
%matplotlib inline

from pathlib import Path
from imutils import paths

from skimage import exposure
from skimage import feature
from sklearn.metrics import confusion_matrix

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop,Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.models import Sequential


from keras.applications.vgg16 import preprocess_input

#from tensorflow.keras.layers import Conv2D, Dense, GlobalAveragePooling2D, Flatten, MaxPool2D


In [81]:
def preprocessing_function(image):
  image = preprocess_input(image)
  img = np.array(image, dtype=np.uint8)
  grayimg = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  #image = cv2.bitwise_not(image)
  #image = cv2.resize(image, (480,360))

  #image = np.stack((image,)*3, axis=-1)
  #image = np.array(image, dtype=np.uint8)

  claheimg = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(9,9))
  img = claheimg.apply(grayimg)


  (H, hogImage) = feature.hog(img, orientations=11, pixels_per_cell=(6,6),cells_per_block=(17,17), transform_sqrt=True, block_norm="L1", visualize=True) 
  hogImage = exposure.rescale_intensity(hogImage, out_range=(0, 255))
  image = hogImage.astype("uint8")
  y = np.expand_dims(image, axis=-1)

  return y

In [82]:
n_classes=6

In [83]:
#Data Aug using Image Data Generator 
 
train_dataset= ImageDataGenerator( 
                        #shear_range = 0.2,
                        validation_split=0.2,
                        #rotation_range=10,
                        brightness_range=[0.2,0.5],
                        #zoom_range=[0.0,0.1],
                        preprocessing_function=preprocessing_function
                        )

validation_dataset = ImageDataGenerator(
                            #shear_range = 0.2,
                            validation_split=0.2,
                            rotation_range=10,
                            #brightness_range=[0.2,0.5],
                            zoom_range=[0.0,0.1],
                            preprocessing_function=preprocessing_function
                            )

In [84]:
print("Train \n",train_dataset.classes)
print('validation \n',validation_dataset.classes)


AttributeError: 'ImageDataGenerator' object has no attribute 'classes'

In [85]:
model = Sequential()
#
model.add(tf.keras.layers.Conv2D(input_shape=(480,360,1),
                 filters=64,  kernel_size=(3,3), padding="same", activation="relu"))
model.add(tf.keras.layers.Dense(units=4096,activation="sigmoid"))
model.add(tf.keras.layers.Dense(units=4096,activation="sigmoid"))
model.add(tf.keras.layers.Dense(units=n_classes, activation="softmax"))

In [ ]:
model = Sequential()
#
model.add(tf.keras.layers.Conv2D(input_shape=(480,360,1),
                 filters=64,  kernel_size=(3,3), padding="same", activation="relu"))

model.add(tf.keras.layers.Conv2D(filters=64,  kernel_size=(3,3), padding="same", activation="relu"))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2)))
#
model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2)))
#
model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2)))
#
model.add(tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2)))
#
model.add(tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2)))
#
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=4096,activation="sigmoid"))
model.add(tf.keras.layers.Dense(units=4096,activation="sigmoid"))
model.add(tf.keras.layers.Dense(units=n_class, activation="softmax"))

In [72]:
tf.keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    name="Adam",
)

In [73]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='categorical_crossentropy',
    metrics=['accuracy'])

In [76]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("model1.h5", 
                             monitor='val_acc', 
                             verbose=1, 
                             save_best_only=True, 
                             save_weights_only=False, 
                             mode='auto', #save_freq=1
                             )

early = EarlyStopping(monitor='val_acc', 
                      min_delta=0, 
                      patience=20, 
                      verbose=1, 
                      mode='auto')

In [ ]:
model_fit =model.fit( 
                      train_dataset,
                      #steps_per_epoch=5,
                      epochs=40, 
                      validation_data= validation_dataset)
                      #callbacks=[checkpoint,early])

In [ ]:
plt.plot(model.history["Accuracy"])
plt.plot(model.history['val_Accuracy'])
plt.plot(model.history['loss'])
plt.plot(model.history['val_loss'])
plt.title("model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy","Validation Accuracy","loss","Validation Loss"])
plt.show()

In [ ]:
#Testing Path 
Results=[]
npval=[]
dir_path='/content/drive/MyDrive/dataset/test'
print(class_names,"\n") 

for i in os.listdir(dir_path):
  img= image.load_img(dir_path+'//'+i,target_size=(450,300))
  print(i)
  plt.imshow(img)
  plt.show()
  x= image.img_to_array(img)
  x=np.expand_dims(x,axis=0)
  #imgs=np.vstack([x])

  val= resnet_model.predict(x)
  npval.append(np.argmax(val))